# 예제

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs=torch.Tensor(1,1,28,28)  #batch, channel, height, width

In [3]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [4]:
conv1=nn.Conv2d(1,32,3,padding=1) # 1-> 32 channel, kernel_size=3, padding=1
pool=nn.MaxPool2d(2) 
conv2=nn.Conv2d(32,64,3,padding=1) # 32->64 channel, kernel_size=3, padding=1

In [5]:
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [7]:
out=conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [8]:
out=pool(out)
out.shape

torch.Size([1, 32, 14, 14])

# MNIST

## 라이브러리 로드

In [9]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

## GPU , random_seed 설정

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
print(device)
torch.cuda.is_available()

cuda


True

## 하이퍼 파라미터

In [12]:
learning_rate=0.01
epochs=15
batch_size=100

## MNIST Dataset

In [13]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                       train=True,
                       transform=transforms.ToTensor(),
                        download=True)

mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

## DataLoader

In [14]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                  batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)

## class CNN

In [15]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [16]:
model= CNN().to(device)

In [17]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [18]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [19]:
total_batch=len(data_loader)

for e in range(epochs):
    avg_cost=0
    
    for image,label in data_loader:
        image=image.to(device) #연산 위해 torch -cuda
        label=label.to(device)
        
        optimizer.zero_grad() # Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문, 1iter 마다 0으로 초기화
        hypothesis=model(image) # 가설
        
        cost=criterion(hypothesis,label)
        cost.backward()
        optimizer.step()
        
        avg_cost+= cost/total_batch
        
    print(f'[Epoch : {e +1 } cost={ avg_cost} ]')
    
print('Learning Finished')

[Epoch : 1 cost=0.17674435675144196 ]
[Epoch : 2 cost=0.07137061655521393 ]
[Epoch : 3 cost=0.059587642550468445 ]
[Epoch : 4 cost=0.05626598373055458 ]
[Epoch : 5 cost=0.05199291929602623 ]
[Epoch : 6 cost=0.04780910536646843 ]
[Epoch : 7 cost=0.04590580239892006 ]
[Epoch : 8 cost=0.04528481513261795 ]
[Epoch : 9 cost=0.04321351274847984 ]
[Epoch : 10 cost=0.041123729199171066 ]
[Epoch : 11 cost=0.04129207879304886 ]
[Epoch : 12 cost=0.03778209164738655 ]
[Epoch : 13 cost=0.038466524332761765 ]
[Epoch : 14 cost=0.037482064217329025 ]
[Epoch : 15 cost=0.03570949286222458 ]
Learning Finished


In [20]:
with torch.no_grad():
    X_test= mnist_test.data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test= mnist_test.targets.to(device)
    
    pre=model(X_test)
    correct=torch.argmax(pre,1)==Y_test
    accuracy=correct.float().mean()
    print(f'Accuracy:{accuracy.item()}')

Accuracy:0.7616999745368958


## Layer3, Layer4,5 추가

In [21]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
            
        # FC1 4*4*128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(3*3*128, 625)
        # Final FC 7x7x64 inputs -> 10 outputs
        self.relu=torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [22]:
model= CNN().to(device)

In [23]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [24]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
value=torch.Tensor(1,1,28,28).to(device)
model(value)

tensor([[-2.4622e+35, -2.3513e+35,  2.1016e+35, -3.1294e+35,  3.0945e+35,
          3.0100e+35,  4.1137e+35, -1.9250e+35,  7.8237e+34,  2.3623e+35]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [26]:
total_batch=len(data_loader)

for e in range(epochs):
    avg_cost=0
    
    for image,label in data_loader:
        image=image.to(device) #연산 위해 torch -cuda
        label=label.to(device)
        
        optimizer.zero_grad() # Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문, 1iter 마다 0으로 초기화
        hypothesis=model(image) # 가설
        
        cost=criterion(hypothesis,label)
        cost.backward()
        optimizer.step()
        
        avg_cost+= cost/total_batch
        
    print(f'[Epoch : {e +1 } cost={ avg_cost} ]')
    
print('Learning Finished')

[Epoch : 1 cost=0.24065932631492615 ]
[Epoch : 2 cost=0.08496139943599701 ]
[Epoch : 3 cost=0.07487007230520248 ]
[Epoch : 4 cost=0.07037733495235443 ]
[Epoch : 5 cost=0.06719613820314407 ]
[Epoch : 6 cost=0.06837671995162964 ]
[Epoch : 7 cost=0.07036831974983215 ]
[Epoch : 8 cost=0.06124445050954819 ]
[Epoch : 9 cost=0.05818888172507286 ]
[Epoch : 10 cost=0.06006084382534027 ]
[Epoch : 11 cost=0.06365584582090378 ]
[Epoch : 12 cost=0.057791296392679214 ]
[Epoch : 13 cost=0.05354469642043114 ]
[Epoch : 14 cost=0.05404578894376755 ]
[Epoch : 15 cost=0.052008550614118576 ]
Learning Finished


In [27]:
with torch.no_grad():
    X_test= mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test= mnist_test.test_labels.to(device)
    
    pre=model(X_test)
    correct=torch.argmax(pre,1)==Y_test
    accuracy=correct.float().mean()
    print(f'Accuracy:{accuracy.item()}')

Accuracy:0.8104999661445618


C:\Users\Eunji\miniconda3\envs\ML\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Eunji\miniconda3\envs\ML\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
